# Reference

This example is taken from the book [DL with Python](https://www.manning.com/books/deep-learning-with-python) by F. Chollet. 

All the notebooks from the book are available for free on [Github](https://github.com/fchollet/deep-learning-with-python-notebooks)

If you like to run the example locally follow the instructions provided on [Keras website](https://keras.io/#installation)

---

In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.2.4'

# Text generation with LSTM

This notebook contains the code samples found in Chapter 8, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff). Note that the original text features far more content, in particular further explanations and figures: in this notebook, you will only find source code and related comments.

----

[...]

## Implementing character-level LSTM text generation


Let's put these ideas in practice in a Keras implementation. The first thing we need is a lot of text data that we can use to learn a 
language model. You could use any sufficiently large text file or set of text files -- Wikipedia, the Lord of the Rings, etc. In this 
example we will use some of the writings of Nietzsche, the late-19th century German philosopher (translated to English). The language model 
we will learn will thus be specifically a model of Nietzsche's writing style and topics of choice, rather than a more generic model of the 
English language.

## Preparing the data

Let's start by downloading the corpus and converting it to lowercase:

In [2]:
import keras
import numpy as np

path = keras.utils.get_file(
    'nietzsche.txt',
    origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('Corpus length:', len(text))

Corpus length: 600901



Next, we will extract partially-overlapping sequences of length `maxlen`, one-hot encode them and pack them in a 3D Numpy array `x` of 
shape `(sequences, maxlen, unique_characters)`. Simultaneously, we prepare a array `y` containing the corresponding targets: the one-hot 
encoded characters that come right after each extracted sequence.

In [3]:
# Length of extracted character sequences
maxlen = 60

# We sample a new sequence every `step` characters
step = 3

# This holds our extracted sequences
sentences = []

# This holds the targets (the follow-up characters)
next_chars = []

for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

# List of unique characters in the corpus
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))
# Dictionary mapping unique characters to their index in `chars`
char_indices = dict((char, chars.index(char)) for char in chars)

# Next, one-hot encode the characters into binary arrays.
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Number of sequences: 200281
Unique characters: 59
Vectorization...


## Building the network

Our network is a single `LSTM` layer followed by a `Dense` classifier and softmax over all possible characters. But let us note that 
recurrent neural networks are not the only way to do sequence data generation; 1D convnets also have proven extremely successful at it in 
recent times.

In [4]:
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

Since our targets are one-hot encoded, we will use `categorical_crossentropy` as the loss to train the model:

In [5]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

## Training the language model and sampling from it


Given a trained model and a seed text snippet, we generate new text by repeatedly:

* 1) Drawing from the model a probability distribution over the next character given the text available so far
* 2) Reweighting the distribution to a certain "temperature"
* 3) Sampling the next character at random according to the reweighted distribution
* 4) Adding the new character at the end of the available text

This is the code we use to reweight the original probability distribution coming out of the model, 
and draw a character index from it (the "sampling function"):

In [6]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)


Finally, this is the loop where we repeatedly train and generated text. We start generating text using a range of different temperatures 
after every epoch. This allows us to see how the generated text evolves as the model starts converging, as well as the impact of 
temperature in the sampling strategy.

In [ ]:
import random
import sys

for epoch in range(1, 60):
    print('epoch', epoch)
    # Fit the model for 1 epoch on the available training data
    model.fit(x, y,
              batch_size=128,
              epochs=1)

    # Select a text seed at random
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index: start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # We generate 400 characters
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200281/200281 [==============================] - 365s 2ms/step - loss: 2.0279
--- Generating with seed: "

=personal change.=--as soon as a religion rules, it has fo"
------ temperature: 0.2


=personal change.=--as soon as a religion rules, it has for the self and the spirit of the sumple the man and a be a godness of the stand and a consequently and a consequent and desiness of the consequently and has a consequent of the spirit and be man and the spirit and indecting of the intring and man and a be a man and and a something and a spiritions of the spirit of the something and indingual and the age and suing of the has a general and and not a
------ temperature: 0.5
ual and the age and suing of the has a general and and not and perhaps and al has somether the spire of the fains so the inderitions of the even in the and one of the only the experience and spirility of the promough of the act and as leated and godness in
the inding at in the moctinity of the a supers an

always "or and shour" of dontful of courdeans, and that sendousned world (and not readceped are no wonlys, through those masteral
"have thikegot to unknowly time his sudectually the xtoul, very insidecition:"-pus it not to from general labution shoritbory and cushced froiccy. forresed--the lagatal that their sigrert "then which
mattiness by the ole
epoch 5
Epoch 1/1
200281/200281 [==============================] - 522s 3ms/step - loss: 1.5005
--- Generating with seed: "othed himself.
passion will not wait: the tragic element in "
------ temperature: 0.2
othed himself.
passion will not wait: the tragic element in the considerations of the fact and all the same them of the more the believed and and the pard to the more and the many and the spirit and more and and and and many and in the concerning the sense of the distrust of the fact the considerations and the problem of the more the many and the fact of the more and all the consequently and the problem of the consequently and the man i

C:\Users\simon\AppData\Local\conda\conda\envs\tf\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


 of the condition, and a spirit and super of the present to the conception of the believe and such a demerity, and the 
------ temperature: 0.5
 the conception of the believe and such a demerity, and the be some to many, he are futher of the desires in the democrent and pride only a perhaps believe in sympathy discovered and differences of the conception, there is deduc«--and a such a desier in liberation with the germand us only the philosopher and every actual instroding strength, for the philosophers there is democration of the constroding perions and fines of a problemed that the might be almo
------ temperature: 1.0
ding perions and fines of a problemed that the might be almost
is plight for
the ous animal-tour example is in (that
sight, in love
takes it: and superation, is alwolick of
very present were a meded thint always a loun!--his brougins are exhaustoric linest, there is are conscience order, modern account of
,of it seems on the severeful velly f¦justity.--he seem of wils 

to be somehow unfair--it is certai©bl©©©" ©©d©f©©qu©y©" and©ju© ©©qu©y©-©©©y who would©©©y and the prove©f--th©y th©y ©y,©quati© of the ©©©w©©w©"©" an©f©©©©©©©©©-©y is©--an©fure the s©qual©""©" to be proper to the more and the mo©bs ©©©©©w©©©
is©--and t©qu©-©©©ws©"-©qu©y©©©©,©©©"©©©"©©©, the partic all the©"©©, and the can be ©quaxity of the prov©©©©©©ws t©©b©y©--as th©©"©©©©©©©© o©, and©qual ©bi©©©©©©" and the© and the p©y ©y the
------ temperature: 0.5
©©©©©©©© o©, and©qual ©bi©©©©©©" and the© and the p©y ©y the contres©" a©qu©jo©y of the enit©-©©©qual©"©©©,©©w and l©©©©©©w©""--and th©w: an©, ©q©©©y in the b©btic but i©©q©©gulation, any©p©©©ura©,©--the contu©y ©q©y©" the©©©©--a perh©©©--an©:©fully wa©t of t©©©©©©ws ©man: but ©gristed ©m©©tvent therefo©©©qu©y of©qual themselves as©©© of©qu©bs©, whi©©"©-©©©g©w, this©""©--a©©w©©©©©©©©©©ju©s, an©tway© that©© in the grea© ©y to be one and personall©,©:©" the 
------ temperature: 1.0
y© that©© in the grea© ©y to be one and personall©,©:©" 


As you can see, a low temperature results in extremely repetitive and predictable text, but where local structure is highly realistic: in 
particular, all words (a word being a local pattern of characters) are real English words. With higher temperatures, the generated text 
becomes more interesting, surprising, even creative; it may sometimes invent completely new words that sound somewhat plausible (such as 
"eterned" or "troveration"). With a high temperature, the local structure starts breaking down and most words look like semi-random strings 
of characters. Without a doubt, here 0.5 is the most interesting temperature for text generation in this specific setup. Always experiment 
with multiple sampling strategies! A clever balance between learned structure and randomness is what makes generation interesting.

Note that by training a bigger model, longer, on more data, you can achieve generated samples that will look much more coherent and 
realistic than ours. But of course, don't expect to ever generate any meaningful text, other than by random chance: all we are doing is 
sampling data from a statistical model of which characters come after which characters. Language is a communication channel, and there is 
a distinction between what communications are about, and the statistical structure of the messages in which communications are encoded. To 
evidence this distinction, here is a thought experiment: what if human language did a better job at compressing communications, much like 
our computers do with most of our digital communications? Then language would be no less meaningful, yet it would lack any intrinsic 
statistical structure, thus making it impossible to learn a language model like we just did.


## Take aways

* We can generate discrete sequence data by training a model to predict the next tokens(s) given previous tokens.
* In the case of text, such a model is called a "language model" and could be based on either words or characters.
* Sampling the next token requires balance between adhering to what the model judges likely, and introducing randomness.
* One way to handle this is the notion of _softmax temperature_. Always experiment with different temperatures to find the "right" one.